# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-25 07:48:04] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32099, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=967271180, constrained_

[2025-06-25 07:48:05] Inferred chat template from model path: llama-2


[2025-06-25 07:48:13] Attention backend not set. Use flashinfer backend by default.
[2025-06-25 07:48:13] Init torch distributed begin.


[2025-06-25 07:48:14] Init torch distributed ends. mem usage=0.00 GB


[2025-06-25 07:48:15] Load weight begin. avail mem=31.64 GB


...

[2025-06-25 07:48:19] Init torch distributed begin.
[2025-06-25 07:48:19] Init torch distributed ends. mem usage=0.00 GB
[2025-06-25 07:48:19] Load weight begin. avail mem=6.84 GB


[2025-06-25 07:48:19] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]

[2025-06-25 07:48:20] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=5.81 GB, mem usage=1.03 GB.
[2025-06-25 07:48:20] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-25 07:48:20] Memory pool end. avail mem=5.49 GB


[2025-06-25 07:48:21] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=5.72 GB
[2025-06-25 07:48:21] INFO:     Started server process [3330275]
[2025-06-25 07:48:21] INFO:     Waiting for application startup.
[2025-06-25 07:48:21] INFO:     Application startup complete.
[2025-06-25 07:48:21] INFO:     Uvicorn running on http://127.0.0.1:32099 (Press CTRL+C to quit)


[2025-06-25 07:48:22] INFO:     127.0.0.1:57116 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-25 07:48:22] INFO:     127.0.0.1:57120 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 07:48:22] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-25 07:49:27] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-25 07:49:57] INFO:     127.0.0.1:57124 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 07:49:57] The server is fired up and ready to roll!


[2025-06-25 07:50:00] INFO:     127.0.0.1:57136 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-25 07:50:07] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=30568, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=123945918, constrained_js

[2025-06-25 07:50:07] Inferred chat template from model path: llama-2


[2025-06-25 07:50:17] Attention backend not set. Use flashinfer backend by default.
[2025-06-25 07:50:17] Init torch distributed begin.


[2025-06-25 07:50:17] Init torch distributed ends. mem usage=0.00 GB


[2025-06-25 07:50:18] Load weight begin. avail mem=30.29 GB


...

[2025-06-25 07:50:22] Init torch distributed begin.
[2025-06-25 07:50:22] Init torch distributed ends. mem usage=0.00 GB
[2025-06-25 07:50:22] Load weight begin. avail mem=5.61 GB
[2025-06-25 07:50:22] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.35s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.35s/it]

[2025-06-25 07:50:24] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=20.85 GB, mem usage=-15.24 GB.
[2025-06-25 07:50:24] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-25 07:50:24] Memory pool end. avail mem=20.50 GB


[2025-06-25 07:50:24] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=20.42 GB
[2025-06-25 07:50:25] INFO:     Started server process [3338403]
[2025-06-25 07:50:25] INFO:     Waiting for application startup.
[2025-06-25 07:50:25] INFO:     Application startup complete.
[2025-06-25 07:50:25] INFO:     Uvicorn running on http://127.0.0.1:30568 (Press CTRL+C to quit)


[2025-06-25 07:50:25] INFO:     127.0.0.1:38106 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-25 07:50:26] INFO:     127.0.0.1:38116 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 07:50:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-25 07:51:16] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-25 07:51:18] INFO:     127.0.0.1:38124 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 07:51:18] The server is fired up and ready to roll!


[2025-06-25 07:51:21] INFO:     127.0.0.1:38136 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-25 07:51:21] Child process unexpectedly failed with exitcode=9. pid=3338615
[2025-06-25 07:51:21] Child process unexpectedly failed with exitcode=9. pid=3338549


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-25 07:51:29] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=31738, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=5130

[2025-06-25 07:51:29] Casting torch.bfloat16 to torch.float16.


[2025-06-25 07:51:36] Casting torch.bfloat16 to torch.float16.


[2025-06-25 07:51:37] Casting torch.bfloat16 to torch.float16.
[2025-06-25 07:51:37] Attention backend not set. Use flashinfer backend by default.
[2025-06-25 07:51:37] Init torch distributed begin.


[2025-06-25 07:51:37] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-25 07:51:55] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-06-25 07:51:55] INFO:     Started server process [3342286]
[2025-06-25 07:51:55] INFO:     Waiting for application startup.
[2025-06-25 07:51:55] INFO:     Application startup complete.
[2025-06-25 07:51:55] INFO:     Uvicorn running on http://127.0.0.1:31738 (Press CTRL+C to quit)


[2025-06-25 07:51:56] INFO:     127.0.0.1:37006 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-25 07:51:56] INFO:     127.0.0.1:37008 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 07:51:56] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 07:51:59] INFO:     127.0.0.1:37020 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 07:51:59] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-25 07:52:01] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 07:52:01] INFO:     127.0.0.1:36002 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-25 07:52:01] Child process unexpectedly failed with exitcode=9. pid=3342799


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-25 07:52:11] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32733, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1030291235, c

[2025-06-25 07:52:11] Casting torch.bfloat16 to torch.float16.


[2025-06-25 07:52:18] Casting torch.bfloat16 to torch.float16.


[2025-06-25 07:52:19] Casting torch.bfloat16 to torch.float16.
[2025-06-25 07:52:19] Attention backend not set. Use flashinfer backend by default.
[2025-06-25 07:52:19] Init torch distributed begin.


[2025-06-25 07:52:19] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.27it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.27it/s]

[2025-06-25 07:52:34] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=58.29 GB, mem usage=1.82 GB.
[2025-06-25 07:52:34] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-06-25 07:52:34] Memory pool end. avail mem=58.21 GB


[2025-06-25 07:52:35] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.89 GB


[2025-06-25 07:52:35] INFO:     Started server process [3343790]
[2025-06-25 07:52:35] INFO:     Waiting for application startup.
[2025-06-25 07:52:35] INFO:     Application startup complete.
[2025-06-25 07:52:35] INFO:     Uvicorn running on http://127.0.0.1:32733 (Press CTRL+C to quit)
[2025-06-25 07:52:35] INFO:     127.0.0.1:46222 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-25 07:52:36] INFO:     127.0.0.1:46236 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 07:52:36] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 07:52:40] INFO:     127.0.0.1:46246 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 07:52:40] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-25 07:52:41] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 07:52:41] INFO:     127.0.0.1:46256 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-25 07:52:49] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=35238, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=493557148, constrained_json_whitespace_pattern=None,

[2025-06-25 07:52:58] Attention backend not set. Use flashinfer backend by default.
[2025-06-25 07:52:58] Init torch distributed begin.


[2025-06-25 07:52:58] Init torch distributed ends. mem usage=0.00 GB


[2025-06-25 07:52:58] Load weight begin. avail mem=78.50 GB


[2025-06-25 07:52:59] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  7.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  7.11it/s]

[2025-06-25 07:53:04] Load weight end. type=MiMoMTP, dtype=torch.bfloat16, avail mem=57.83 GB, mem usage=2.71 GB.
[2025-06-25 07:53:04] KV Cache is allocated. #tokens: 20480, K size: 1.41 GB, V size: 1.41 GB
[2025-06-25 07:53:04] Memory pool end. avail mem=54.88 GB


[2025-06-25 07:53:04] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-25 07:53:05] INFO:     Started server process [3344809]
[2025-06-25 07:53:05] INFO:     Waiting for application startup.
[2025-06-25 07:53:05] INFO:     Application startup complete.
[2025-06-25 07:53:05] INFO:     Uvicorn running on http://0.0.0.0:35238 (Press CTRL+C to quit)


[2025-06-25 07:53:06] INFO:     127.0.0.1:43820 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-25 07:53:06] INFO:     127.0.0.1:43834 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 07:53:06] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 07:53:08] INFO:     127.0.0.1:43850 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 07:53:08] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-25 07:53:11] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 07:53:11] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, accept len: 1.69, cuda graph: False, gen throughput (token/s): 9.87, #queue-req: 0


[2025-06-25 07:53:12] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, accept len: 1.77, cuda graph: False, gen throughput (token/s): 113.49, #queue-req: 0


[2025-06-25 07:53:13] Decode batch. #running-req: 1, #token: 228, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 108.70, #queue-req: 0


[2025-06-25 07:53:13] Decode batch. #running-req: 1, #token: 292, token usage: 0.01, accept len: 1.60, cuda graph: False, gen throughput (token/s): 103.15, #queue-req: 0


...

[2025-06-25 07:53:15] Decode batch. #running-req: 1, #token: 427, token usage: 0.02, accept len: 1.70, cuda graph: False, gen throughput (token/s): 105.27, #queue-req: 0


[2025-06-25 07:53:15] Decode batch. #running-req: 1, #token: 489, token usage: 0.02, accept len: 1.55, cuda graph: False, gen throughput (token/s): 97.95, #queue-req: 0


[2025-06-25 07:53:16] Decode batch. #running-req: 1, #token: 554, token usage: 0.03, accept len: 1.62, cuda graph: False, gen throughput (token/s): 102.39, #queue-req: 0


[2025-06-25 07:53:17] Decode batch. #running-req: 1, #token: 624, token usage: 0.03, accept len: 1.75, cuda graph: False, gen throughput (token/s): 113.08, #queue-req: 0


[2025-06-25 07:53:17] INFO:     127.0.0.1:43862 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-25 07:53:17] Child process unexpectedly failed with exitcode=9. pid=3345021


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).